In [1]:
# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
import time
# Local webdriver functions
from src.data.setup_webdriver import initialize_webdriver
from src.data.goodreads_login import login
# Goodreads User ID scraping functions
from src.data.scrape_users import gather_user_ids
from src.data.save_user_ids import save_raw_ids
from src.data.adjust_shelf import adjust_shelf_settings
from src.data.scroll_down import scroll_to_bottom
from src.data.read_review_elements import get_review
from src.data.scrape_shelf import scrape_user_read_shelf
# Other imports
from configparser import ConfigParser
import polars as pl
import numpy as np
import pickle
from src import log
import time
import os
import re

In [2]:
user_ids_path = '../data/raw/user_ids.pkl'

In [3]:
def load_user_ids(filepath):
    with open(filepath, 'rb') as file:
        user_ids = pickle.load(file)
    return user_ids

In [4]:
user_ids = load_user_ids(user_ids_path)
print(f'There are {len(user_ids)} IDs, the first 5 are {user_ids[0:5]}.')

There are 316 IDs, the first 5 are ('33674708', '3990914', '51224285', '25683251', '4717304').


In [5]:
chrome_tools_path = '../chrome_tools'

In [6]:
# Configure driver
driver = initialize_webdriver(chrome_tools_path)

Successfully launched webdriver


In [7]:
# Find credentials
config = ConfigParser()
config.read('../src/data/config.ini')
username = config.get('credentials', 'username')
password = config.get('credentials', 'password')

In [8]:
authenticated_driver = login(driver, username, password)

Opened Goodreads successfully.
Reached sign-in options page.
Reached sign-in with email page.
Login attempted.
Login successful.


In [9]:
test = scrape_user_read_shelf(authenticated_driver, 78924283)

Opened shelf settings.
Settings adjusted. Closed settings panel.
All items loaded.


In [11]:
test.head(-10)

cover_url,title,author,isbn,isbn13,asin,avg_rating,user_rating,user_id
str,str,str,str,str,str,f64,i64,i32
"""https://i.gr-a…","""The Devil's Fl…","""Yokomizo, Seis…","""1782278842""","""9781782278849""","""B0BYTSXCG6""",3.91,3,78924283
"""https://i.gr-a…","""Peter and the …","""Barry, Dave""","""078684907X""","""9780786849079""","""078684907X""",4.04,3,78924283
"""https://i.gr-a…","""Blood of Elves…","""Sapkowski, And…","""031602919X""","""9780316029193""","""031602919X""",4.1,4,78924283
"""https://i.gr-a…","""The Hustler (E…","""Tevis, Walter""","""1560254734""","""9781560254737""","""1560254734""",4.12,4,78924283
"""https://i.gr-a…","""The Man Who Fe…","""Tevis, Walter""","""0593467477""","""9780593467473""","""0593467477""",4.06,4,78924283
"""https://i.gr-a…","""Cannery Row (C…","""Steinbeck, Joh…","""014200068X""","""9780142000687""","""014200068X""",4.05,5,78924283
"""https://i.gr-a…","""Sword of Desti…","""Sapkowski, And…","""0316389706""","""9780316389709""","""0316389706""",4.27,4,78924283
"""https://i.gr-a…","""1984""","""Orwell, George…","""""","""""","""""",4.19,0,78924283
"""https://i.gr-a…","""The Two Prince…","""Levine, Gail C…","""0062253549""","""9780062253545""","""""",4.05,3,78924283
